In [1]:
import sys
import gc

#numpy and pandas for data manipulation
import numpy as np
import pandas as pd

#sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

In [2]:
app = pd.read_csv('data/application_train.csv')
bureau = pd.read_csv('data/bureau.csv')
bureau_balance = pd.read_csv('data/bureau_balance.csv')
previous_app = pd.read_csv('data/previous_application.csv')
credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
pos_cash_balance = pd.read_csv('data/POS_CASH_balance.csv')
installments_payments = pd.read_csv('data/installments_payments.csv')

In [3]:
# Create an anomalous flag column
app['DAYS_EMPLOYED_ANOM'] = app["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# Treating as outlier and removing
app = app[app['CODE_GENDER'] != 'XNA']

app['CREDIT_INCOME_PERCENT'] = app['AMT_CREDIT'] / app['AMT_INCOME_TOTAL']
app['ANNUITY_INCOME_PERCENT'] = app['AMT_ANNUITY'] / app['AMT_INCOME_TOTAL']
app['CREDIT_TERM'] = app['AMT_ANNUITY'] / app['AMT_CREDIT']
app['DAYS_EMPLOYED_PERCENT'] = app['DAYS_EMPLOYED'] / app['DAYS_BIRTH']

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app:
    if app[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app[col].unique())) <= 2:
            # Train on the training data
            le.fit(app[col])
            # Transform 
            app[col] = le.transform(app[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
print('%d columns were label encoded.' % le_count)

4 columns were label encoded.


In [4]:
def replace_day_outliers(df):
    """Replace 365243 with np.nan in any columns with DAYS"""
    for col in df.columns:
        if "DAYS" in col:
            if len(df[df[col]==365243]) > 0:
                df[col] = df[col].replace({365243: df[col].mean()})
                df[col+'_ANOM'] = df[col] == 365243
    return df

bureau = replace_day_outliers(bureau)
bureau_balance = replace_day_outliers(bureau_balance)
previous_app = replace_day_outliers(previous_app)
credit_card_balance = replace_day_outliers(credit_card_balance)
pos_cash_balance = replace_day_outliers(pos_cash_balance)
installments_payments = replace_day_outliers(installments_payments)

In [5]:
round(app.memory_usage().sum() / 1e9, 2)

0.31

In [6]:
def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

app = convert_types(app, True)
bureau = convert_types(bureau, True)
bureau_balance = convert_types(bureau_balance, True)
previous_app = convert_types(previous_app, True)
credit_card_balance = convert_types(credit_card_balance, True)
pos_cash_balance = convert_types(pos_cash_balance, True)
installments_payments = convert_types(installments_payments, True)

Original Memory Usage: 0.31 gb.
New Memory Usage: 0.14 gb.
Original Memory Usage: 0.23 gb.
New Memory Usage: 0.1 gb.
Original Memory Usage: 0.66 gb.
New Memory Usage: 0.25 gb.
Original Memory Usage: 0.5 gb.
New Memory Usage: 0.17 gb.
Original Memory Usage: 0.71 gb.
New Memory Usage: 0.34 gb.
Original Memory Usage: 0.64 gb.
New Memory Usage: 0.29 gb.
Original Memory Usage: 0.87 gb.
New Memory Usage: 0.44 gb.


In [7]:
round(app.memory_usage().sum() / 1e9, 2)

0.14

In [8]:
def aggregate_numeric_data(df, parent_var, df_name):
    """
    Groups and aggregates the numeric values in a child dataframe
    by the parent variable.
    
    Parameters
    --------
        df (dataframe): 
            the child dataframe to calculate the statistics on
        parent_var (string): 
            the parent variable used for grouping and aggregating
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated by the `parent_var` for 
            all numeric columns. Each observation of the parent variable will have 
            one row in the dataframe with the parent variable as the index. 
            The columns are also renamed using the `df_name`. Columns with all duplicate
            values are removed. 
    
    """
    
    # Remove id variables other than grouping variable
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Only want the numeric variables
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Need to create new column names
    columns = []

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        if var != parent_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg



def aggregate_categorical_data(df, parent_var, df_name):
    """
    Aggregates the categorical features in a child dataframe
    for each observation of the parent variable.
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    parent_var : string
        The variable by which to group and aggregate the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with aggregated statistics for each observation of the parent_var
        The columns are also renamed and columns with duplicate values are removed.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('category'))

    # Make sure to put the identifying id on the column
    categorical[parent_var] = df[parent_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    # Remove duplicate columns by values
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

print("Application features : {}".format(len(app.columns)))

bureau_agg_num = aggregate_numeric_data(bureau.drop(columns = ['SK_ID_BUREAU']), 
                                        parent_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg_cat = aggregate_categorical_data(bureau.drop(columns = ['SK_ID_BUREAU']),
                                   parent_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_balance_agg_num = aggregate_numeric_data(bureau_balance, parent_var = 'SK_ID_BUREAU',
                                                df_name = 'bureau_balance')
bureau_balance_agg_cat = aggregate_categorical_data(bureau_balance, parent_var = 'SK_ID_BUREAU',
                                                df_name = 'bureau_balance')
monthly_bureau_data = bureau_balance_agg_num.merge(bureau_balance_agg_cat,right_index=True,
                                                   left_on='SK_ID_BUREAU' ,how='outer')
monthly_bureau_data = monthly_bureau_data.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], 
                                               on='SK_ID_BUREAU',how='left')
client_bureau_balance = aggregate_numeric_data(monthly_bureau_data.drop(columns = ['SK_ID_BUREAU']),
                                    parent_var = 'SK_ID_CURR', df_name = 'client')


app = app.merge(bureau_agg_num, on = 'SK_ID_CURR', how = 'left')
app = app.merge(bureau_agg_cat, on = 'SK_ID_CURR', how = 'left')
app = app.merge(client_bureau_balance, on = 'SK_ID_CURR', how = 'left')
previous_app_agg_num = aggregate_numeric_data(previous_app, 'SK_ID_CURR', 'previous')
previous_app_agg_cat = aggregate_categorical_data(previous_app, 'SK_ID_CURR', 'previous')
app = app.merge(previous_app_agg_num, on ='SK_ID_CURR', how = 'left')
app = app.merge(previous_app_agg_cat, on = 'SK_ID_CURR', how = 'left')




Application features : 127


In [9]:
print(previous_app_agg_num.shape)
print(previous_app_agg_cat.shape)
print(client_bureau_balance.shape)

(338857, 80)
(338857, 285)
(134542, 85)


In [10]:
def aggregate_by_client(df, group_vars, df_names):
    """Aggregate a dataframe with data at the loan level 
    at the client level
    
    Args:
        df (dataframe): data at the loan level
        group_vars (list of two strings): grouping variables for the loan 
        and then the client (example ['SK_ID_PREV', 'SK_ID_CURR'])
        names (list of two strings): names to call the resulting columns
        (example ['cash', 'client'])
        
    Returns:
        df_client (dataframe): aggregated numeric stats at the client level. 
        Each client will have a single row with all the numeric data aggregated
    """
    
    # Aggregate the numeric columns
    df_agg = aggregate_numeric_data(df, parent_var = group_vars[0], df_name = df_names[0])
    
    # If there are categorical variables
    if any(df.dtypes == 'category'):
    
        # Aggregate the categorical columns
        df_counts = aggregate_categorical_data(df, parent_var = group_vars[0], df_name = df_names[0])

        # Merge the numeric and categorical
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Merge to get the client id in dataframe
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Aggregate numeric stats by column
        df_by_client = aggregate_numeric_data(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])

        
    # No categorical variables
    else:
        # Merge to get the client id in dataframe
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Remove the loan id
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
        
        # Aggregate numeric stats by column
        df_by_client = aggregate_numeric_data(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])
        
    # Memory management
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

credit_card_bal_agg = aggregate_by_client(credit_card_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                  df_names = ['credit', 'client'])
pos_cash_bal_agg = aggregate_by_client(pos_cash_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                  df_names = ['cash', 'client'])
installments_agg = aggregate_by_client(installments_payments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], 
                                  df_names = ['installments', 'client'])
app = app.merge(credit_card_bal_agg, on = 'SK_ID_CURR', how = 'left')
app = app.merge(pos_cash_bal_agg, on = 'SK_ID_CURR', how = 'left')
app = app.merge(installments_agg, on = 'SK_ID_CURR', how = 'left')

In [11]:
app.shape

(307507, 1332)

In [12]:
id_columns = [x for x in app.columns if 'SK_ID_CURR' in x or 'SK_ID_PREV' in x or 'SK_ID_BUREAU' in x]

In [13]:
app.shape

(307507, 1332)

In [14]:
app = app.drop(columns=id_columns)

In [15]:
app.shape

(307507, 1331)

In [16]:
app.to_csv('training.csv',index=False)